In [4]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import re

In [5]:

slow = 25 * 60  # 25 minutes in seconds
fast = 25  # 25seconds

# Read the metadata file
dfmeta = pd.read_excel("/home/poyraden/Analysis/JOSIEfiles/JOSIE-96-02/Josie_2002_metadata.xls")

for i in range(len(dfmeta)):
    if(dfmeta.at[i,'SondeTypeNr'] < 2): dfmeta.at[i,'ENSCI'] = 0
    if(dfmeta.at[i,'SondeTypeNr'] == 0): dfmeta.at[i,'ENSCI'] = 1
    if(dfmeta.at[i,'SST_Nr'] == 1):
        dfmeta.at[i,'Sol'] = 1.0
        dfmeta.at[i,'Buf'] = 1.0
    if(dfmeta.at[i,'SST_Nr'] == 2):
        dfmeta.at[i,'Sol'] = 0.5
        dfmeta.at[i,'Buf'] = 0.5
    if(dfmeta.at[i,'SST_Nr'] == 3):
        dfmeta.at[i,'Sol'] = 2.0
        dfmeta.at[i,'Buf'] = 0.0
    if(dfmeta.at[i,'SST_Nr'] == 4):
        dfmeta.at[i,'Sol'] = 1.0
        dfmeta.at[i,'Buf'] = 0.1
    if(dfmeta.at[i,'SST_Nr'] == 5):
        dfmeta.at[i,'Sol'] = 2.0
        dfmeta.at[i,'Buf'] = 0.1

In [6]:
dfmeta

,JOSIE_Nr,Sim_Nr,R1_Tstart,R1_Tstop,R2_Tstart,R2_Tstop,GAW_Report_Nr_Details,Part_Nr,SondeTypeNr,SST_Nr,Data_FileName,ENSCI,Sol,Buf
0,2002,112,1000,1160,3710,3880,SAGE_III_Page28,1,1,1,JS02SN01.DS0,0.0,1.0,1.0
1,2002,112,1000,1160,3710,3880,SAGE_III_Page28,2,1,1,JS02SN02.DS0,0.0,1.0,1.0
2,2002,112,1000,1160,3710,3880,SAGE_III_Page28,3,1,1,JS02SN03.DS0,0.0,1.0,1.0
3,2002,112,1000,1160,3710,3880,SAGE_III_Page28,4,1,1,JS02SN04.DS0,0.0,1.0,1.0
4,2002,113,980,1150,3625,3780,SAGE_III_Page28,5,1,1,JS02SN05.DS0,0.0,1.0,1.0
5,2002,113,980,1150,3625,3780,SAGE_III_Page28,6,1,1,JS02SN06.DS0,0.0,1.0,1.0
6,2002,113,980,1150,3625,3780,SAGE_III_Page28,7,1,1,JS02SN07.DS0,0.0,1.0,1.0
7,2002,113,980,1150,3625,3780,SAGE_III_Page28,8,1,1,JS02SN08.DS0,0.0,1.0,1.0
8,2002,114,980,1150,3640,3800,SAGE_III_Page28,9,1,1,JS02SN09.DS0,0.0,1.0,1.0
9,2002,114,980,1150,3640,3800,SAGE_III_Page28,10,1,1,JS02SN10.DS0,0.0,1.0,1.0


In [16]:

# all files
## use pathlib jspr
filenames = dfmeta.Data_FileName.tolist()
path = '/home/poyraden/Analysis/JOSIEfiles/JOSIE-96-02/2002/JOSIE-2002-DS0 Data/Js02-ds0/'
filenames = [path + i for i in filenames]
filenamespath = [Path(j) for j in filenames]

list_data = []

#Some declarations

columnMeta  = ['JOSIE_Nr', 'Sim_Nr', 'R1_Tstart', 'R1_Tstop', 'R2_Tstart', 'R2_Tstop', 'GAW_Report_Nr_Details',
               'Part_Nr' , 'SondeTypeNr', 'SST_Nr', 'Data_FileName', 'ENSCI', 'Sol', 'Buf']

columnString = "Rec_Nr Time_Day Time_Sim Pres_ESC Temp_ESC Temp_Inlet Alt_Sim PO3_OPM I_ECC_RAW Temp_ECC Cur_Motor PO3_ECC_RAW" \
               " PO3_ECC_BG1 PO3_ECC_BG2 PO3_ECC_BG3 PO3_ECC_BG4 Validity_Nr"
columnStr = columnString.split(" ")
columnStr

['Rec_Nr',
 'Time_Day',
 'Time_Sim',
 'Pres_ESC',
 'Temp_ESC',
 'Temp_Inlet',
 'Alt_Sim',
 'PO3_OPM',
 'I_ECC_RAW',
 'Temp_ECC',
 'Cur_Motor',
 'PO3_ECC_RAW',
 'PO3_ECC_BG1',
 'PO3_ECC_BG2',
 'PO3_ECC_BG3',
 'PO3_ECC_BG4',
 'Validity_Nr']

In [19]:





#**********************************************
# Main loop to merge all data sets
#**********************************************
for filename in filenamespath:
    file = open(filename, 'r', encoding="ISO-8859-1")
    infolist = file.readlines()[0:50]
    sim = int(infolist[4].split("*")[0])
    team = int(infolist[5].split("*")[0])
    PFcor = float(infolist[11].split("*")[0])
    print(sim, team, PFcor)

    df = pd.read_csv(filename, engine="python", sep="\s+", skiprows=53, names=columnStr)
    #     ,  encoding = "ISO-8859-1"

    #     # Add the header information to the main df
    df = df.join(pd.DataFrame(
        [[sim, team,  PFcor]],
        index=df.index,
        columns=['Sim', 'Team', 'PFcor']
    ))

#     # Get the index of the metadata that corresponds to this Simulation Number and Participant (Team)

#     select_indicesTeam = list(np.where(dfmeta["Part_Nr"] == df['Team'][0]))[0]
#     select_indicesSim = list(np.where(dfmeta["Sim_Nr"] == df['Sim'][0]))[0]

#     common = [i for i in select_indicesTeam if i in select_indicesSim]
#     index_common = common[0]

#     list_md = dfmeta.iloc[index_common, :].tolist()

#     ## Add  metadata to the main df
#     df = df.join(pd.DataFrame(
#         [list_md],
#         index=df.index,
#         columns=columnMeta
#     ))

#     ## now convert variables to usual Josie naming conventions

#     df['PO3'] = df['PO3_ECC_BG3']
#     df['IM'] = df['I_ECC_RAW']
#     df['TPint'] = df['Temp_ECC']
#     df['Pair'] = df['Pres_ESC']
#     df['Tsim'] = df['Time_Sim']

#     print(df['PO3_OPM'].dtypes, df['PFcor'].dtypes, df['TPint'].dtypes )

#     ## convert OPM pressure to current
#     df['I_OPM'] = (df['PO3_OPM'] * df['PFcor']) / (df['TPint'] * 0.043085)

#     Pval = np.array([1000, 730, 535, 382, 267, 185, 126, 85, 58, 39, 26.5, 18.1, 12.1, 8.3, 6])
#     JMA = np.array(
#         [0.999705941, 0.997216654, 0.995162562, 0.992733959, 0.989710199, 0.985943645, 0.981029252, 0.974634364,
#          0.966705137, 0.956132227, 0.942864263, 0.9260478, 0.903069813, 0.87528384, 0.84516337])

#     for k in range(len(df)):
#         ## jma corrections for OPM current, OPM_I_jma will be used only for Ua in the convolution of
#         ## the slow component of the signal
#         for p in range(len(JMA) - 1):
#             if (df.at[k, 'Pair'] >= Pval[p + 1]) & (df.at[k, 'Pair'] < Pval[p]):
#                 # print(p, Pval[p + 1], Pval[p ])
#                 df.at[k, 'I_OPM_jma'] = df.at[k, 'PO3_OPM'] * df.at[k, 'PFcor'] * JMA[p] / \
#                                           (df.at[k, 'TPint'] * 0.043085)
#                 # df.at[k,'PO3_jma'] = 0.043085 * df.at[k, 'TPint']  * (df.at[k, 'IM'] - df.at[k, 'IB1']) / (df.at[k, 'PFcor'] * JMA[p])
#         if (df.at[k, 'Pair'] <= Pval[14]):
#             df.at[k, 'OPM_I_jma'] = df.at[k, 'PO3_OPM'] * df.at[k, 'PFcor'] * JMA[14] / \
#                                           (df.at[k, 'TPint'] * 0.043085)
#             # df.at[k, 'PO3_jma'] = 0.043085 * df.at[k, 'TPint'] * (df.at[k, 'IM'] - df.at[k, 'IB1']) / (
#             #             df.at[k, 'PFcor'] * JMA[14])



#     size = len(df)
#     Ums_i = [0] * size
#     Ua_i = [0] * size
#     Ums_i[0] = df.at[0, 'IM']


#     ## only convolute slow part of the signal, which is needed for beta calculation
#     for i in range(size-1):

#         Ua_i = df.at[i + 1, 'I_OPM_jma']
#         t1 = df.at[i + 1,'Tsim']
#         t2 = df.at[i,'Tsim']
#         Xs = np.exp(-(t1 - t2) / slow)
#         Xf = np.exp(-(t1 - t2) / fast)
#         Ums_i[i + 1] = Ua_i - (Ua_i - Ums_i[i]) * Xs

#     df['I_conv_slow'] = Ums_i

    list_data.append(df)
#

    #  end of the allfiles loop    #
     
# Merging all the data files to df

df = pd.concat(list_data,ignore_index=True)

df.to_csv("/home/poyraden/Analysis/JOSIEfiles/Proccessed/Josie2002_Data.csv")


    







112 1 220.898
112 2 213.487
112 3 214.762
112 4 223.314
113 1 221.74
113 2 219.088
113 3 212.522
113 4 218.719
114 1 211.175
114 2 218.856


/home/poyraden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [20]:
df

,Alt_Sim,Cur_Motor,I_ECC_RAW,PFcor,PO3_ECC_BG1,PO3_ECC_BG2,PO3_ECC_BG3,PO3_ECC_BG4,PO3_ECC_RAW,PO3_OPM,Pres_ESC,Rec_Nr,Sim,Team,Temp_ECC,Temp_ESC,Temp_Inlet,Time_Day,Time_Sim,Validity_Nr
0,0.000000,86.4453,0.761091,NaN,2.68237,2.68241,2.57524,2.57537,2.71809,2.54168,996.24400,1,NaN,NaN,305.276,294.060,293.952,39593.0,0.0,1
1,0.007257,86.4414,0.757283,NaN,2.66881,2.66887,2.56167,2.56192,2.70452,2.55063,995.40400,2,NaN,NaN,305.280,294.074,294.005,39595.5,2.5,1
2,0.014927,86.4374,0.748519,NaN,2.63754,2.63764,2.53040,2.53078,2.67326,2.59791,994.51800,3,NaN,NaN,305.284,294.078,294.046,39598.0,5.0,1
3,0.027397,86.4335,0.764347,NaN,2.69410,2.69425,2.58696,2.58755,2.72982,2.64509,993.07700,4,NaN,NaN,305.288,293.979,293.932,39600.5,7.5,1
4,0.039872,86.4295,0.755626,NaN,2.66299,2.66319,2.55585,2.55664,2.69871,2.65475,991.63800,5,NaN,NaN,305.292,293.880,293.819,39603.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57355,32.918200,92.7868,1.480380,218.856,5.52614,5.56346,5.41339,5.56266,5.21157,5.47050,7.07700,2864,114.0,2.0,298.146,224.576,225.113,44591.5,7157.5,1
57356,32.925000,92.7945,1.483260,218.856,5.53715,5.57447,5.42439,5.57367,5.22169,5.50007,7.06960,2865,114.0,2.0,298.144,224.576,225.090,44594.0,7160.0,1
57357,32.931900,92.7907,1.430730,218.856,5.33985,5.37717,5.22710,5.37638,5.03669,5.52966,7.06220,2866,114.0,2.0,298.140,224.576,225.066,44596.5,7162.5,1
57358,32.936900,92.7868,1.450820,218.856,5.41544,5.45276,5.30268,5.45197,5.10735,5.55895,7.05686,2867,114.0,2.0,298.136,224.609,225.115,44599.0,7165.0,1
